# Het bouwen van een Simpel aanbevelingssysteem voor Films

# 1. Business vraagstuk
*Data River Movies* is een online film platform en wil net zich profileren als de beste film aanbieder. Het huidige platform heeft een groot assortiment aan films, maar gebruikers blijven niet lang op het platform. 

Sterker nog, na het kijken van een film zijn de gebruikers weg. Dit wil *Data River Movies* graag verbeteren. Ze hebben gehoord over een 'recommendation system' en vragen of jij hun hiermee kunt helpen. 

## 1.2 Recommendation Systems
De snelle groei van dataverzameling heeft geleid tot een nieuw informatietijdperk. Data worden gebruikt om efficiëntere systemen te creëren en dit is waar *Recommendation Systems* (of aanbevelingssystemen) in het spel komen. Recommendation Systems zijn een soort informatie filtersystemen omdat zij de kwaliteit van zoekresultaten verbeteren en items aanbieden die relevanter zijn voor het zoekitem of gerelateerd zijn aan de zoekgeschiedenis van de gebruiker. De systemen worden gebruikt om de waardering of voorkeur te voorspellen die een gebruiker aan een item zou geven.


# 2. Data Understanding
Nu we weten wat de business wil, kunnen we beginnen met het verzamelen van de data. Gelukkig is deze overzichtelijk bijeen gebracht in een groot csv bestand

* tmdb_5000_movies.csv

Deze bevat alle gegevens van de 5000 films waarvoor een systeem gemaakt moet worden. Laten we beginnen met de eerste stap van het CRISP-DM model, *Data Understanding*. 

De eerste en gemakkelijkste stap is om de data even te downloaden en te openen. In dit geval is het een csv bestand dat niet heel groot is en handmatig kan worden bekeken. Op die manier kun je alvast bekend worden met de data waar we mee gaan werken. 

## 2.1. Data inlezen

In [1]:
# Importeren van de benodige packages
import os
import pandas as pd 

In [2]:
# Opgeven van de hoofdmap
root_dir = ""

# Opgeven van de project map
project_folder = "Casus"     # ZELF AANPASSEN NAAR EIGEN MAP


In [4]:
# Methode om de project map te maken als die niet bestaat
# En om je werkmap aan te passen naar de project map 
def create_and_set_working_directory(project_folder):
  # Controleren of de map bestaan en anders aanmaken
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # Aanpassen van de werkmap naar de project map
  os.chdir(root_dir + project_folder)

  print('\nYour working directory was changed to ' + root_dir + project_folder )
  

In [5]:
# Aanroepen van de methode om je werkmap aan te passen naar je project map
# Dat maakt het lezen van de bestanden een stuk eenvoudiger
create_and_set_working_directory(project_folder)



Your working directory was changed to Casus


De basis instellingen staan nu goed. 

Laten we nu de data inlezen en bekijken. 

In [7]:
# Inlezen van de datasets
# Movie dataset
df = pd.read_csv('tmdb_5000_movies.csv')

# Bekijken van de eerste 10 regels van de movie datasetd
display(df.head(X))                                                       # ZELF AANPASSEN      

# Bekijken van de laatste X regels van de moviedataset
display(df.tail(X))


NameError: name 'X' is not defined

## 2.2 Data bekijken en begrijpen
Alle bestanden samen vormen de movies dataset.  De volgende kolommen komen voor in de verschillende datasets: 

* budget - Het totale budget om de film te maken
* genre - Het genre waarin de film valt, zoals Action, Comedy ,Thriller etc.
* homepage - Een link naar de website van de film
* id - De unieke ID van de film
* keywords - De sleutelwoorden of tags van de film
* original_language - De orignele taal van de film
* original_title - De originele titel van de film
* overview - Een korte beschrijving van de film
* popularity - Een nummerieke waarde die de populatiteit van de film aangeeft.
* production_companies - Het filmhous die de film heeft gemaakt
* production_countries - Het land waarin de film is gemaakt
* release_date - De datum waarop de film is uitgekomen
* revenue - De wereldwijde opbrengsten van de film
* runtime - De speeltijd in minuten
* status - Al uitgebracht "Released" of verwacht "Rumored"
* tagline - Movie's tagline.
* title - Titel van de film
* vote_average - Gemiddelde rating
* vote_count - Totaal aantal stemmen
  

## 2.3 Basis statistieken
Laten we als eerste kijken naar de film dataset en wat eerste statistieken. Een standaad functie helpt ons hierbij, het zogenaamde *beschrijven* van de dataset.

In [ ]:
# Als eerste kunnen we de numerieke waarden van de filmset bekijken
display(df.describe())


Na deze overzichten hebben we nu een goed beeld van de aanwezige data. Volgende stap is dan natuurlijk de data voorbereiden om verder te kunnen met het bouwen van een *Recommendation System*. 

# 3 Data Preparation
Als eerste viel op dat een aantal kolommen een vreemde waarde hebben. Dit gaan we als eerste oplossen. 

# 3.1 Kolomwaarden goed zetten
Als eerste een functie om de waarden uit te lezen en om te zetten. De waarden zitten nu in een dictionary en moeten goed worden gezet om het lezen ervan gemakkelijker te maken. Dit wordt gedaan door de *split_dictionary* functie. 


In [ ]:
def split_dictionary(i_df, i_split_column, i_id, i_name): 
  # Loop door de bestaande items om ze op te slaan in een lijst
  new_values = []
  
  for i in i_df[i_split_column]:
    dict_list = eval(i)
    items = [] 

    for j in dict_list:
      items.append(j[i_name])
    new_values.append(items)
  return new_values

In [ ]:
# Pas de kolommen naar leesbare waarden
df['genres_extra'] = split_dictionary(df, 'genres', 'id', 'name')
df['spoken_languages_extra'] = split_dictionary(df, 'spoken_languages', 'iso_639_1', 'name')

display(df.head(3))

Kun jij dit doen voor de andere kolommen? Let goed op hoe je de functie moet gebruiken! 

In [ ]:
# Pas de kolommen
df['production_companies_extra'] = split_dictionary(df, 'XXX', 'XXX', 'XXX')    # ZELF AANPASSEN
df['production_countries_extra'] = split_dictionary(df, 'XXX', 'XXX', 'XXX')    # ZELF AANPASSEN
df['keywords_extra'] = split_dictionary(df, 'XXX', 'XXX', 'XXX')                # ZELF AANPASSEN


Voordat we verder kunnen met het recommendation sytem hebben we nog een aantal maatstaven nodig die nu niet in de data zit. Denk hierbij aan

* een maatstaaf om films te scoren of te beoordelen
* een de score voor elke film
* de scores sorteren en de best beoordeelde film aanbevelen aan de gebruikers.

## 3.2 Best beoordeelde film
We kunnen de gemiddelde waardering van de film als score gebruiken, maar dit is niet eerlijk. Een film met een gemiddelde waardering van 8,9 en slechts 3 stemmen is niet beter per definitie beter dan een film met een gemiddelde waardering van 7,8 en 140 stemmen. 

Voor een eerlijke gewogen gemiddelde kunnen we de IMDB's gewogen score gebruiken. De formule daarvoor is als volgt: 

\begin{equation}
Weighted Rating (WR) = (\frac{v}{v+m}\times R) + (\frac{m}{v+m} \times C)
\end{equation}
waar 
* v is het aantal stemmen voor de film;
* m is het minimum aantal stemmen dat nodig is om in de grafiek te worden opgenomen;
* R is de gemiddelde waardering van de film; en
* C is het gemiddeld aantal stemmen over de gehele dataset

Het aantal stemmen (v) is al aanwezig in de dataset, net al het gemiddelde aantal stemmen (R). C kunnen we gemakkelijk berekenen en m kunnen we zelf bepalen aan de hand van statistiek. 

In [ ]:
# Het berekenen van de gemiddelde stem
C = df['XXX'].mean()                                                            # ZELF AANPASSEN
print(f'Het gemiddelde aantal stemmen over de gehele dataset is: {C}')

In [ ]:
# Het berekenen van het minimum aantal stemmen dat nodig is
# Hiervoor nemen we aan dat 10% van de data buiten de normaalverdeling valt en dus voor ruis veroorzaakt. 
# Hier mag je uiteraard van afwijken. 
m = df['XXX'].quantile(0.9)                                                     # ZELF AANPASSEN
print(f'Het minimum aantal stemmen dat nodig is om meegenomen te worden is: {m}')

In [ ]:
# Selecteren van de films die voldoende stemmen hebben. Hiervoor geldt dat het aantal stemmen (vote_count) groter of gelijk moet zijn aan m
qualified_movies = df.copy().loc[df['XXX'] >= m]                                # ZELF AANPASSEN
# Door de shape te bekijken zien we het aantal rijen (films) en het aantal kolommen (eigenschappen)
print(qualified_movies.shape)
print(f'In totaal zijn er {qualified_movies.shape[0]} films geselecteeds met voldoende stemmen')


Nu alle elementen van de formule om het de gewogen score te berekenen, kunnen we de formule toepassen. Om te voorkomen dat we de formule meerdere keren moeten uitschrijven, maken we een functie om de berekening gemakkelijk voor de gehele dataset uit te rekenen. 

In [ ]:
# Functie voor het berekenen van het gewogen gemiddelde
def weighted_rating(x, m = m, C = C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
# Aan de gefilterde dataset voegen we de berekende score toe als nieuwe kolom. 
qualified_movies['score'] = qualified_movies.apply(weighted_rating, axis=1)

# Nette notatie:
qualified_movies['score'] = weighted_rating(qualified_movies)

In [ ]:
# We kunnen de data sorteren op de hoogste score om de beste film te achterhalen
qualified_movies = qualified_movies.sort_values('XXX', ascending=False)         # ZELF AANPASSEN

# Weergeven van de top 10 meest gewaardeerde films
qualified_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

Hoera de basis is gezet voor het recommendation system. Op basis van de gewogen waardering kunnen we al betere **algemene** suggesties doen aan de gebruikers van *River data Movies*. 

## 3.3 Meest populaire film 
Laten we nu ook kijken naar de populaire films, dit noemen we ook wel de *trending* films. Afhankelijk van de tijd zal dit veranderen. De dataset heeft al een kolom met de populariteit van een film. 


In [ ]:
populaire_films_df = df.copy().sort_values('XXX', ascending=False)              # ZELF AANPASSEN
populaire_films_df[['title', 'XXX']].head(10)                                   # ZELF AANPASSEN

In [ ]:
# Het maken van een plot met horizontale balken
populaire_films_df[['title', 'popularity']].head(10).plot(kind = 'barh', x = 'title', y = 'popularity')

Dat ging goed en was eigenlijk best makkelijk toch? 

#4 Modeling
Het overzicht van de populaire films, zoals we dat nu gedaan hebben, is een algemeen overzicht. Nu willen we wel op basis van de specifieke interesses van de gebruiker films aanraden. 

Laten we nu gaan kijken naar een iets beter systeem om meer in te kunnen spelen op de voorkeuren van de gebruiker. 

#4.1 Inhoud gebaseerd filteren
Laten we kijken op basis van de inhoud van een film (overzicht, cast, crew, trefwoord, slogan enz.) of we op basis daarvan soortgelijke films kunnen vinden. Zo worden er films aangeraden die het meeste overeenkomen. Dit kunnen we doen op basis van de filmbeschrijvingen. 

Laten we eerst kijken naar de beschrijvingen van de films.

In [ ]:
display(df[['title', 'XXX']].head(5))                                           # ZELF AANPASSEN

display(df['overview'][0])

Voor het analyseren van de inhoud moeten we een stukje text mining toepassing. Dit doen we door de woorden terug te brengen naar de stam om ze zo goed te kunnen vergelijken en er berekeningen op los te laten. 

##4.2 Woord frequentie
We beginnen met het bekijken van de woorden en hoevaak deze voorkomen in de gehele tekst, ook bekend als *Term Frequency* (TF). De *Inverse Document Frequency* (IDF) is het aantal teksten waarin de termen voorkomen en wordt berekend als log(aantal documenten/documenten met term). Het totale belang van elk woord voor de documenten waarin ze voorkomen is gelijk aan TF * IDF

Dit levert een matrix op waarbij elke kolom een woord uit de woordenschat is (alle woorden die in minstens één document voorkomen) en elke rij een film. Dit doen we om het belang van woorden die vaak voorkomen in plotoverzichten te verminderen en dus ook hun betekenis bij het berekenen van de uiteindelijke overeenkomst score.

Gelukkig hoef je dit niet allemaal zelf te doen. Hiervoor zijn packages beschikbaar en wij gaan die van *scikit-learn* gebruiken. 


In [ ]:
# Import TfIdfVectorizer van scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Definieer een TF-IDF Vectorizer Object
# Verwijder de engelse stopwoorden zoals'the', 'a'
tfidf = TfidfVectorizer(stop_words = 'english')

# Vervang alle lege waardne met een lege sting --> NaN naar lege string
df['overview_extra'] = df['overview'].fillna('')

# Maak de TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(df['overview_extra'])

# Bekijk de vorm van de tfidf_matrix
tfidf_matrix.shape

Gezamenlijk bevatten alle filmbeschrijvingen meer dan 20.000 verschillende woorden om de 4800 films in de dataset te beschrijven.

### 4.2.1 Berekenen van de overeenkomst score
De eerste stap voor het berekenen van gelijke films is nu gezet. Het bereken kan op verschillende manieren, zoals de euclidische, de Pearson en de cosinus overeenkomst score. Er is geen juist antwoord op de vraag welke score de beste is, dat verschilt per dataset. Mocht je dit zelf willen gaan doen, is het nadig om meerdere berekeningen te gebruiken en zo de beste te kiezen. 

Wij zullen de **cosinus** gebruiken om een numerieke waarde te berekenen die de gelijkheid tussen twee films aangeeft. We gebruiken de cosinus overeenkomst score omdat die onafhankelijk is van de grootte en relatief gemakkelijk en snel te berekenen is. De formule hiervoor is als volgt: 

\begin{equation}
cos(\Theta) = \frac{\sum_{i=1}^{n} A_{i} B_{i}}{\sqrt{\sum_{i=1}^{n} A_{i}^{2}} \sqrt{\sum_{i=1}^{n} B_{i}^{2}}}
\end{equation}

Een deel van de functie hebben we al dankzij de TF-IDF waardoor het vermenigvuldigen ons direct de overeenkomst score geeft.  


In [ ]:
# Import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

# Berekenen van de cosinus overeenkomstscore matrix
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Bekijken van de vorm van de matrix (zou vierkant moeten zijn)
cos_sim.shape

Het meeste werk is nu gedaan. De laatste stap is om een functie te maken om aan de hand van een film titel aanbevelingen te krijgen voor andere films die erop lijken. 

### 4.2.2 Recommendation function
Dit doen we door de functie *get_recommendation* te maken, waarbij je een film titel mee moet geven voor gelijkwaardige suggesties. 

Om een goede suggestie te krijgen, zijn er een aantal stappen die door de functie worden gedaan. 



In [ ]:
# Functie voor het krijgen van suggesties van soorgelijke films
def get_recommendations(title, i_df = df, cosine_sim=cos_sim):
    # Maak een opzoek tabel op basis van de titel
    indices = pd.Series(i_df.index, index = i_df['title']).drop_duplicates()
    
    # Vind de film in de dataset aan de hand van de titel
    idx = indices[title]

    # Bereken de overeenkomst score tussen alle films en de opgegeven film
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sorteer de films op volgorde van de score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Selecteerd de top 10 films met de hoogste score
    sim_scores = sim_scores[1:11] 

    # Bepalen welke titels het zijn
    movie_indices = [i[0] for i in sim_scores]

    # Teruggeven van de top 10 most overeenkomende films
    return i_df['title'].iloc[movie_indices]

Gefeliciteerd! 

We hebben onze eigen *recommendation system* gemaakt die de populaire films kan weergeven, maar ook op basis van de fimbeschrijving een aanbeveling kan doen. 

# 5 Oefening
# 5.1 Gebruik het recommendation system
Kun jij nu uitzoeken wat goede suggesties zijn voor de volgende films? 
* Avatar
* Brave
* Seventh Son
* Ghost Rider
* Superman Returns
* The Curious Case of Benjamin Button
* How to Train Your Dragon 2
* Contact
* The Lord of the Rings: The Fellowship of the Ring
* Night at the Museum


In [ ]:
print(f'De films die het meeste overeenkomen met XXX')                          # ZELF AANPASSEN
display(get_recommendations('XXX'))                                             # ZELF AANPASSEN

print(f'De films die het meeste overeenkomen met XXX')                          # ZELF AANPASSEN
display(get_recommendations('XXX'))                                             # ZELF AANPASSEN


In [ ]:
print(f'De films die het meeste overeenkomen met XXX')                          # ZELF AANPASSEN
display(get_recommendations('XXX'))                                             # ZELF AANPASSEN

print(f'De films die het meeste overeenkomen met XXX')                          # ZELF AANPASSEN
display(get_recommendations('XXX'))                                             # ZELF AANPASSEN


In [ ]:
print(f'De films die het meeste overeenkomen met XXX')                          # ZELF AANPASSEN
display(get_recommendations('XXX'))                                             # ZELF AANPASSEN

print(f'De films die het meeste overeenkomen met XXX')                          # ZELF AANPASSEN
display(get_recommendations('XXX'))                                             # ZELF AANPASSEN


In [ ]:
print(f'De films die het meeste overeenkomen met XXX')                          # ZELF AANPASSEN
display(get_recommendations('XXX'))                                             # ZELF AANPASSEN

print(f'De films die het meeste overeenkomen met XXX')                          # ZELF AANPASSEN
display(get_recommendations('XXX'))                                             # ZELF AANPASSEN


In [ ]:
print(f'De films die het meeste overeenkomen met XXX')                          # ZELF AANPASSEN
display(get_recommendations('XXX'))                                             # ZELF AANPASSEN

print(f'De films die het meeste overeenkomen met XXX')                          # ZELF AANPASSEN
display(get_recommendations('XXX'))                                             # ZELF AANPASSEN


## 5.2 Genres van de films
En kun je ook de genres vinden in de dataset die horen bij de aanbevolen films? 

<details>
<summary>Klik hier voor een hint</summary>
De functie <i>get_recommendation_genre</i> heeft een kleine aanpassing nodig op de laatste regel.
</details>

In [ ]:
# Functie voor het krijgen van suggesties van soorgelijke films
def get_recommendations_genre(title, i_df = df, cosine_sim=cos_sim):
    # Maak een opzoek tabel op basis van de titel
    indices = pd.Series(i_df.index, index = i_df['title']).drop_duplicates()
    
    # Vind de film in de dataset aan de hand van de titel
    idx = indices[title]

    # Bereken de overeenkomst score tussen alle films en de opgegeven film
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sorteer de films op volgorde van de score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Selecteerd de top 10 films met de hoogste score
    sim_scores = sim_scores[1:11]

    # Bepalen welke titels het zijn
    movie_indices = [i[0] for i in sim_scores]

    # Teruggeven van de top 10 most overeenkomende films
    return i_df[['title', 'XXX']].iloc[movie_indices]


In [ ]:
print(f'De films die het meeste overeenkomen met XXX')                          # ZELF AANPASSEN
display(get_recommendations_genre('XXX'))                                       # ZELF AANPASSEN
